In [3]:
import os, sys
import numpy as np
import pandas as pd
from pathlib import Path

# define the current path (notebooks in lab_utils)
currpath = os.getcwd()
labutilspath = str(Path(currpath).parents[1])
sys.path.append(labutilspath)

# import the autoscan routines
from autoscan import autoscan

In [4]:
# define paths
basepath = '/home/nuburu/sandbox/data/characterization/'

# set the asdatapath accordingly (where is the atuoscan data inside basedatapath?)
asdatapath   = 'autoscan'

# set datapath
datapath = os.path.join(basepath, asdatapath)

pp = autoscan.postprocess(labutilspath = labutilspath)
pp.debug = False

In [5]:
# load the summary dataframe with all measurement
summary = pd.read_csv(os.path.join(datapath,'summary.csv'))

# remove ztop and zbottom
# t = summary['relroot'].apply(lambda x: len(re.findall(r'ztop|zbottom', x))==0)
# summary = summary.loc[t,:].copy()

# shortern column's names
summary.rename(columns = pp.info_columns_short, inplace = True)

# set side to a numeric value (this makes it easy to save data and avoid performance hits due to dtypes mismatchs)
summary.side.fillna('a', inplace = True)
summary.side = summary.side.apply(pp._val_replace)

# set s0 as the default subsample when none is defined
summary.subtag.replace(np.nan, 's0', inplace = True) 

# sort the columns in an specific order
sortcols = ['tag','subtag', 'instance', 'side', 'probe']
summary.sort_values(by = sortcols, ignore_index = True, inplace = True)

fdesc = summary.set_index(['family', 'code'] + sortcols[:-1]).loc[:, ['probe', 'relroot']].copy()
fdesc = pp._fix_repeated_probes(fdesc, debug = False)

print('\n####### summary #######\n', fdesc.describe().iloc[[0, 1, 3], :], '\n', sep = '\n')
# fdesc.info(verbose = False)
N = fdesc.index.nunique(dropna = False)

print('number of unique samples: %d' % (N))

[]
all good

####### summary #######

       probe relroot
count    749     749
unique     4     749
freq     222       1


number of unique samples: 249


In [6]:
dfl = []
db = pp.empty_dataframe()
key_names = fdesc.index.names + ['m']
fdesc['loaded'] = False

for k, r in enumerate(fdesc.index.unique()):
    ftest = fdesc.loc[[r], :]
    print("loaded: %2.2f" % (100 * k / N), r, sep = '\t', end = '\r')
    try:
        df = pp.read_sample_data(ftest, datapath = datapath)        
        dfl.append(df)
        del df
        fdesc.loc[[r], 'loaded'] = True
    except:
        print('\n\ncheck ', fdesc.loc[[r], 'relroot'])
df = pd.concat(dfl)

df.describe()

,x,y,l_1,l_2,l_3,l_4,l_5,l_6,l_7,l_8,...,l_1749,l_1750,l_1751,l_1752,e_star,perm,vp_0,vs_0,vp_1,vs_1
count,69858.000000,69858.000000,45109.000000,45109.000000,45109.000000,45109.000000,45109.000000,45109.000000,45109.000000,45109.000000,...,45109.000000,45109.000000,45109.000000,45109.000000,27170.000000,4.105000e+04,42187.000000,42187.000000,2.465900e+04,2.465900e+04
mean,116.433057,61.079740,2.206719,2.288839,2.383208,2.454404,2.469428,2.452037,2.432671,2.417937,...,2.138742,2.138171,2.137363,2.136565,19.131374,1.262674e+03,4017.648283,2617.985560,6.247876e+03,2.686786e+04
std,133.272734,61.559122,0.520469,0.515309,0.510911,0.510780,0.515536,0.522515,0.529164,0.536344,...,0.604827,0.604586,0.603905,0.603592,11.944766,4.905413e+04,1883.492838,4019.730304,5.456477e+04,2.694143e+05
min,0.000000,0.000000,0.703060,0.842830,0.852320,0.853350,0.855100,0.856620,0.858810,0.856000,...,1.426810,1.428680,1.429260,1.427610,0.486411,4.698540e-01,330.519000,-89552.238806,-1.217391e+06,-3.181818e+06
25%,27.000000,12.000000,2.013930,2.094130,2.183020,2.252040,2.268350,2.248060,2.221560,2.197040,...,1.727950,1.727790,1.727420,1.727160,12.464500,1.765512e+00,3180.806107,1849.201984,2.794597e+03,1.852060e+03
50%,70.000000,40.000000,2.246980,2.335100,2.434310,2.508970,2.527180,2.508310,2.482640,2.457210,...,1.980540,1.980130,1.979120,1.978390,15.612700,2.181990e+00,3824.614124,2486.457686,4.542849e+03,2.585920e+03
75%,150.000000,100.000000,2.430290,2.509550,2.606650,2.688610,2.710140,2.696060,2.682840,2.673090,...,2.301300,2.301210,2.300150,2.299140,22.335100,6.027870e+01,4741.556924,2819.416381,4.913804e+03,2.780941e+03
max,782.000000,500.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,...,6.000000,6.000000,6.000000,6.000000,119.882000,5.307500e+06,69953.364424,7526.477071,4.468085e+06,3.500000e+07


In [11]:
# # this is needed because level 5 (side) has a mix of str and int. The best would be to fix it from the start in summary
# df.reset_index(drop = False, inplace = True)
# df.side = df.side.apply(val_replace)
# df.set_index(keys = key_names, inplace = True)

ftir
perm


In [7]:
# save tje data to an h5 file, mode ='w' means it will overwrite the file there. 
savefile = os.path.join(datapath, 'autoscan.h5')
df.to_hdf(savefile, key = 'data', mode = 'w', data_columns = True, complevel = 9)
fdesc.to_hdf(savefile, key = 'description',  mode = 'a')